In [1]:
# import libraries
import keras
import pandas as pd
import numpy as np
import theano as th
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

from keras.models import Sequential
from keras.layers import Dense


import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\ProgramData\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (thean

In [2]:
# load Dataset
data = pd.read_csv('Churn_Modelling.csv', index_col= 1)
data.drop(['RowNumber','Surname'], axis = 1, inplace =True)

data = pd.get_dummies(data = data,columns =  ['Geography','Gender'], drop_first = True, dtype = float)

X = data.drop('Exited',axis = 1)
y = data['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)


std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.fit_transform(X_test)

Now, if I train artifical neural network on same or different test set then accuracy vary. This is known as variance. Having large variance is not good as we can't be certain about result. So, A good approach is to use cross validation. But cross validation is a module in scikit learn and ANN is built on Keras so its important to combine these two technologies and we will use basic wrapper concept of programming. We will import wrapper module of Keras and built a function that return the artictecture of artifical neural network.

In [3]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.layers import Dropout

In [4]:
def ann_architecture():
    clf = Sequential()
    clf.add(units = 6, input_dim = 11, activation = 'relu', kernel_initializer = 'uniform')
    #clf.add(Dropout(p = 0.1))
    clf.add(Dense(units = 6, activation = 'relu', kernel_initializer = 'uniform'))
    #clf.add(Dropout(p = 0.1))

    clf.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'uniform'))

    clf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )
    return clf


In [ ]:
classifier = KerasClassifier(build_fn= ann_architecture, batch_size= 10, epochs = 100)
accuracies = cross_val_score(estimator= classifier, cv = 10, X = X_train, y = y_train, n_jobs= -1)
mean = accuracies.mean()
variance = accuracies.std()

##### Dropout

Dropout concept in ANN is similar to regularization in machine learning. If artifical neural network is overtrained and it learns too much from the training data and performs poor on test set or it we can say it has high variance. Then Dropout is the saver. 

What dropout does basically is it disable some of the neurons in one or multiple layers randomly on each training iteration. When we train neural network then neurons learns different correlation from dataset independently and depends upon each other for complete picture. 

Suppose there are two neurons a and b. a learns the correlation between age( independent variable) and income(dependent variable) and b learn correlation between salary and income. Now, neurons get activated in the layers according to the conditions. If confition related to age get satisfied then neuron a get activated and transfer its output ti next layer. This is basic functioning of neurons in neural network structure.

When dropout randomly disable some neurons on each trianing iteration then neurons learn correlations more independently and dependence of neurons on each other reduces and overfitting reduces.

For ex. 3 students learn 3 chapters of book and copied from each other in exam and get 100% marks  but failed to perform individualy or some questions of slightly different pattern( in dataset of slightly different correlation). But if we randomly separate the students from each other then they will learn more independently and will perform better almost same in all exams. In datas cience terms, output accuracy will be more or less same on all test sets and variance will be low. Hence, less or No overfitting.

Dropout can be applied to one or multiple layer and its better to apply dropout in the starting layers so that problem of overfitting arise less. Value of P is choosen such that p = 0.1  means 10% of no of neurons in that layer. if no of neurons  = 10 then p = 0.1 means one neuron will be disable. 

#### TIP: start with p = 0.1 and if overfitting still persists then try 0.2, 0.3, 0.4 but try not to go beyond 0.4 because then a great number of neurons will be disabled and underfitting problem takes place.

#### Tuning ANN

In [5]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def ann_architecture(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn = ann_architecture)
parameters = {'batch_size': [15, 25, 32],
             'epochs': [100, 500],
             'optimizer':['adam','rmsprop']
             }
grid = GridSearchCV(estimator= classifier, param_grid= parameters, scoring= 'accuracy', cv = 10)
grid = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_


Epoch 1/100
7200/7200 [==============================] - 3s 402us/step - loss: 0.5174 - acc: 0.7922
Epoch 2/100
7200/7200 [==============================] - 1s 87us/step - loss: 0.4354 - acc: 0.7933
Epoch 3/100
7200/7200 [==============================] - 1s 130us/step - loss: 0.4299 - acc: 0.7933
Epoch 4/100
7200/7200 [==============================] - 1s 90us/step - loss: 0.4264 - acc: 0.7933
Epoch 5/100
7200/7200 [==============================] - 1s 90us/step - loss: 0.4227 - acc: 0.8171
Epoch 6/100
7200/7200 [==============================] - 1s 91us/step - loss: 0.4206 - acc: 0.8239
Epoch 7/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4185 - acc: 0.8261
Epoch 8/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.4171 - acc: 0.8283
Epoch 9/100
7200/7200 [==============================] - 1s 88us/step - loss: 0.4162 - acc: 0.8301
Epoch 10/100
7200/7200 [==============================] - 1s 86us/step - loss: 0.4148 - acc: 0.8303
Epoch 

7200/7200 [==============================] - 1s 91us/step - loss: 0.4026 - acc: 0.8332
Epoch 83/100
7200/7200 [==============================] - 1s 88us/step - loss: 0.4026 - acc: 0.8322
Epoch 84/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.4026 - acc: 0.8342
Epoch 85/100
7200/7200 [==============================] - 1s 90us/step - loss: 0.4026 - acc: 0.8344
Epoch 86/100
7200/7200 [==============================] - 1s 84us/step - loss: 0.4024 - acc: 0.8328
Epoch 87/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4023 - acc: 0.8353
Epoch 88/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.4026 - acc: 0.8347
Epoch 89/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4022 - acc: 0.8336
Epoch 90/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4019 - acc: 0.8346
Epoch 91/100
7200/7200 [==============================] - 1s 89us/step - loss: 0.4029 - acc: 0.8347
Epoch 92/100


7200/7200 [==============================] - 1s 90us/step - loss: 0.4022 - acc: 0.8365
Epoch 64/100
7200/7200 [==============================] - 1s 89us/step - loss: 0.4019 - acc: 0.8340
Epoch 65/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4022 - acc: 0.8354
Epoch 66/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4017 - acc: 0.8353
Epoch 67/100
7200/7200 [==============================] - 1s 89us/step - loss: 0.4020 - acc: 0.8353
Epoch 68/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4019 - acc: 0.8344
Epoch 69/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.4016 - acc: 0.8322
Epoch 70/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4020 - acc: 0.8333
Epoch 71/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.4015 - acc: 0.8354
Epoch 72/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.4016 - acc: 0.8365
Epoch 73/100


7200/7200 [==============================] - 1s 96us/step - loss: 0.3616 - acc: 0.8531
Epoch 45/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.3589 - acc: 0.8538
Epoch 46/100
7200/7200 [==============================] - 1s 95us/step - loss: 0.3566 - acc: 0.8549
Epoch 47/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3544 - acc: 0.8564
Epoch 48/100
7200/7200 [==============================] - 1s 111us/step - loss: 0.3531 - acc: 0.8542
Epoch 49/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.3517 - acc: 0.8558
Epoch 50/100
7200/7200 [==============================] - 1s 134us/step - loss: 0.3501 - acc: 0.8560
Epoch 51/100
7200/7200 [==============================] - 1s 108us/step - loss: 0.3495 - acc: 0.8569
Epoch 52/100
7200/7200 [==============================] - 1s 104us/step - loss: 0.3481 - acc: 0.8586
Epoch 53/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.3471 - acc: 0.8582
Epoch 54

7200/7200 [==============================] - 1s 99us/step - loss: 0.3954 - acc: 0.8388
Epoch 26/100
7200/7200 [==============================] - 1s 93us/step - loss: 0.3951 - acc: 0.8400
Epoch 27/100
7200/7200 [==============================] - 1s 101us/step - loss: 0.3943 - acc: 0.8389
Epoch 28/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.3949 - acc: 0.8400
Epoch 29/100
7200/7200 [==============================] - 1s 97us/step - loss: 0.3949 - acc: 0.8389
Epoch 30/100
7200/7200 [==============================] - 1s 107us/step - loss: 0.3946 - acc: 0.8399
Epoch 31/100
7200/7200 [==============================] - 1s 94us/step - loss: 0.3939 - acc: 0.8404
Epoch 32/100
7200/7200 [==============================] - 1s 99us/step - loss: 0.3940 - acc: 0.8407
Epoch 33/100
7200/7200 [==============================] - 1s 103us/step - loss: 0.3941 - acc: 0.8407
Epoch 34/100
7200/7200 [==============================] - 1s 114us/step - loss: 0.3939 - acc: 0.8413
Epoch 35/

...

7200/7200 [==============================] - 1s 117us/step - loss: 0.3483 - acc: 0.8558
Epoch 67/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3483 - acc: 0.8564
Epoch 68/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3482 - acc: 0.8586
Epoch 69/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3482 - acc: 0.8560
Epoch 70/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3473 - acc: 0.8578
Epoch 71/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3464 - acc: 0.8560
Epoch 72/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3473 - acc: 0.8569
Epoch 73/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3474 - acc: 0.8547
Epoch 74/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3480 - acc: 0.8574
Epoch 75/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3465 - acc: 0.8588
Epo

7200/7200 [==============================] - 1s 117us/step - loss: 0.3446 - acc: 0.8590
Epoch 147/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3453 - acc: 0.8576
Epoch 148/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3451 - acc: 0.8589
Epoch 149/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3457 - acc: 0.8583
Epoch 150/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3452 - acc: 0.8589
Epoch 151/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3454 - acc: 0.8590
Epoch 152/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3453 - acc: 0.8579
Epoch 153/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3456 - acc: 0.8579
Epoch 154/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3461 - acc: 0.8578
Epoch 155/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3438 - acc: 0

7200/7200 [==============================] - 1s 116us/step - loss: 0.3453 - acc: 0.8581
Epoch 227/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3463 - acc: 0.8575
Epoch 228/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3452 - acc: 0.8572
Epoch 229/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3452 - acc: 0.8596
Epoch 230/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3451 - acc: 0.8588
Epoch 231/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3447 - acc: 0.8563
Epoch 232/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3447 - acc: 0.8581
Epoch 233/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3450 - acc: 0.8576
Epoch 234/500
7200/7200 [==============================] - 1s 118us/step - loss: 0.3453 - acc: 0.8585
Epoch 235/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3451 - acc: 0

7200/7200 [==============================] - 1s 116us/step - loss: 0.3450 - acc: 0.8600
Epoch 307/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3440 - acc: 0.8603
Epoch 308/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3450 - acc: 0.8571
Epoch 309/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3449 - acc: 0.8581
Epoch 310/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3452 - acc: 0.8567
Epoch 311/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3435 - acc: 0.8596
Epoch 312/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3448 - acc: 0.8557
Epoch 313/500
7200/7200 [==============================] - 1s 118us/step - loss: 0.3435 - acc: 0.8594
Epoch 314/500
7200/7200 [==============================] - 1s 117us/step - loss: 0.3448 - acc: 0.8586
Epoch 315/500
7200/7200 [==============================] - 1s 120us/step - loss: 0.3444 - acc: 0

5265/7200 [====================>.........] - ETA: 0s - loss: 0.3447 - acc: 0.8587